In [1]:
# prompt: Import libraries

import pandas as pd
import numpy as np
import re
import datetime
import matplotlib.pyplot as plt


In [ ]:
# Load the pickled DataFrame
# complete_data = pd.read_pickle("C:\\Users\\dfber\\OneDrive - Mass General Brigham\\Epidural project\\Data\\processed_merlin_data.pkl")
complete_data = pd.read_pickle("C:\\Users\\User\\OneDrive - Mass General Brigham\\Epidural project\\Data\\processed_merlin_data.pkl")

# Now you can work with the DataFrame
complete_data.head()

In [3]:
df = complete_data.copy()

In [2]:
s = pd.Series([20, 21, 12],
              index=['London', 'New York', 'Helsinki'])

In [ ]:
s

In [13]:
def subtract_custom_value(x, custom_value=5):
    return x - custom_value

In [ ]:
subtract_custom_value(5)

In [ ]:
s.apply(lambda x: subtract_custom_value(x))

In [ ]:
s.apply(subtract_custom_value, custom_value=10)

# Manually analyze some successes and failures

In [ ]:
raw_identified_data = pd.read_csv("C:\\Users\\User\OneDrive - Mass General Brigham\\Epidural project\\Data\\Full Identified raw anesthesia_procedure_notes.csv")

In [ ]:
raw_identified_data.loc[raw_identified_data['NoteID'] == '2362576456']

In [6]:
# Create a mapping of 'NoteID' to 'epic_pmrn' from raw_identified_data
note_to_mrn = raw_identified_data.set_index('NoteID')['epic_pmrn'].to_dict()
note_to_purpose = raw_identified_data.set_index('NoteID')['NotePurposeDSC'].to_dict()

# Use the mapping to create the new 'mrn' column in df
df['epic_pmrn'] = df['anes_procedure_note_id_2260'].map(note_to_mrn)
df['NotePurposeDSC'] = df['anes_procedure_note_id_2260'].map(note_to_purpose)

In [7]:
# prompt: Choose 10 random failed_catheters and 10 random non-failed_catheters

# Assuming 'df' is your DataFrame and it contains a column 'failed_catheter'
failed_catheters = df[df['failed_catheter'] == 1]
non_failed_catheters = df[df['failed_catheter'] == 0]

# Randomly choose 10 failed catheters
random_failed_catheters = failed_catheters.sample(n=10, random_state=42)  # random_state for reproducibility
chosen_failed_catheter_encounter_ids = ['3324914343','3272008150','3234765502','3305371022','3216449190','3186345033','3493903332','3285273066','3320528828','3191160118']
chosen_failed_catheters = df[df['anes_procedure_encounter_id_2273'].isin(chosen_failed_catheter_encounter_ids)]

# Randomly choose 10 non-failed catheters
random_non_failed_catheters = non_failed_catheters.sample(n=10, random_state=42) # random_state for reproducibility

In [8]:
column_names = [
    'epic_pmrn',
    "best_timestamp",
    "failed_catheter",
    "true_procedure_type",
    "NotePurposeDSC",
    "Regulated_Anesthesiologist_Name",
    "Regulated_Resident_Name",
    "anes_procedure_encounter_id_2273",
    "anes_procedure_note_id_2260",
    "subsequent_proof_of_failure_note_id",
]

In [ ]:
df['true_procedure_type'].value_counts()

In [ ]:
random_failed_catheters[column_names]

In [ ]:
chosen_failed_catheters[column_names]

In [ ]:
random_non_failed_catheters[column_names]

In [ ]:
df[df['anes_procedure_encounter_id_2273'] == '3191160118'][column_names]

In [14]:
# Filter the DataFrame for failed catheters and delivery location 'mgh'
random_failed_catheters_mgh = df[(df['failed_catheter'] == True) & (df['delivery_site_2188'] == 'mgh')].sample(n=10, random_state=42)

# Display the chosen sample
random_failed_catheters_mgh[column_names]
chosen_failed_catheters_mgh_encounter_ids = ["3268447806", "3396191507", "3258959083", "3581696894", "3271964781", "3583787789", "3402989492", "3476124055", "3304131417", "3522418740"]
chosen_failed_catheter_mgh_note_ids = ['2903598031', '6426160113', '2535157730', '11282242570',
       '3002237621', '11340428769', '6612736939', '8559605944',
       '3947063203', '9788012155']

In [ ]:
df[df['anes_procedure_encounter_id_2273'].isin(chosen_failed_catheters_mgh_encounter_ids)][column_names]

In [ ]:
df[df['anes_procedure_note_id_2260'].isin(chosen_failed_catheter_mgh_note_ids)][column_names]

In [ ]:
random_failed_catheters_mgh[column_names]

# Look at cases where a failure is replaced by the same anesthesia team

Encounter_ID 3607123568 is an example where the same attending/resident team did the index procedure and the replacement (in this case because the epidural migrated out)

In [18]:
def check_if_replacement_has_same_anesthesia_team(row, df):
    if row['is_neuraxial_catheter'] and row['failed_catheter']:
        this_anesthesiologist = row['Regulated_Anesthesiologist_Name']
        this_resident = row['Regulated_Resident_Name']
        subsequent_procedures = row['subsequent_proof_of_failure_note_id'].replace('\'','').replace('[','').replace(']','').split(',')
        for note_id in subsequent_procedures:
            replacing_anesthesiologist = df[(df['anes_procedure_note_id_2260'] == note_id) & (df['is_neuraxial_catheter'] == True)]['Regulated_Anesthesiologist_Name']
            replacing_resident = df[(df['anes_procedure_note_id_2260'] == note_id) & (df['is_neuraxial_catheter'] == True)]['Regulated_Resident_Name']
            if (this_anesthesiologist == replacing_anesthesiologist).any() and (this_resident == replacing_resident).any():
                # print()
                # print('new note')
                # print(this_anesthesiologist)
                # print(this_resident)
                # print(replacing_anesthesiologist)
                # print(replacing_resident)
                # print(this_anesthesiologist == replacing_anesthesiologist)
                # print(this_resident == replacing_resident)
                row['replaced_by_same_team'] = True


In [ ]:
df[df['failed_catheter']].apply(lambda x: check_if_replacement_has_same_anesthesia_team(x, df), axis=1)